In [24]:
#同义词替换
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords#引入停用词，因为对停用词进行数据增强相当于没有增强
from nltk.corpus import wordnet as wn#引入同义词
import random
stop_words=stopwords.words('english')
for w in ['!',',','.','?','-s','-ly','</s>','s']:
    stop_words.append(w)
   
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def synonym_replacement(words):
    new_words = words.copy()
    n=len(words)/4
    random_word_list = list(word for word in words if word not in stop_words)     
    random.shuffle(random_word_list)
    num_replaced = 0  
    for random_word in random_word_list:          
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)   
            new_words = [synonym if word == random_word else word for word in new_words]   
            num_replaced += 1
        if num_replaced >= n: 
            break

    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return " ".join(new_words)

#获取同义词 wn.synset('apple.n.01').lemma_names()
def get_synonyms(word):
  synonyms = []
  for syn in wn.synsets(word):
    for lm in syn.lemmas():
             synonyms.append(lm.name())
    return synonyms

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
########################################################################
# 随机插入
# 随机在语句中插入n个词
########################################################################
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def random_insertion(words):
    new_words = words.copy()
    n=len(new_words)/n
    for _ in range(n):
        add_word(new_words)
    return " ".join(new_words)
#插入单词，这里插入随机挑选的单词的同义词
def add_word(new_words):
    synonyms = []
    counter = 0    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = random.choice(synonyms)
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)


In [5]:
########################################################################
# 随机交换
# 随机交换几次
########################################################################
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def random_swap(words):
    new_words = words.copy()
    n=len(new_words)/4
    for _ in range(n):
        new_words = swap_word(new_words)
    return " ".join(new_words)

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words


In [6]:
########################################################################
# 随机删除
# 以概率p删除语句中的词
########################################################################
#这里传入的words是一个列表,
#eg:"hello world".split(" ") or ["hello","world"]
def random_deletion(words):

    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > 0.25:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return " ".join(new_words)

In [7]:
def getEda(df,func):
    title=df.title_en.tolist()
    abstract=df.abstract_en.tolist()
    assignee=df.assignee_en.tolist()
    label_id=df.label_id.tolist()
    len1=len(title)
    exTitle=[]
    exass=[]
    exabs=[]
    exlabel=[]
    for i in range(len1):
        lentitle=len(title[i])
        lenassignee=len(assignee[i])
        lenabstract=len(abstract[i])
        txtitle=func(title[i].split(" "))
        txass=func(assignee[i].split(" "))
        txabs=func(abstract[i].split(" "))
        if i%1000==0:
            print(i)
        exTitle.append(txtitle)
        exass.append(txass)
        exabs.append(txabs)
        exlabel.append(label_id[i])  
    return exTitle,exass,exabs,exlabel

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
#读取数据
import pandas as pd
import numpy as np
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/train_en.csv',index_col=0)

In [30]:
edaData=df.sample(frac=1.0)
edaData.head()

,id,title,assignee,abstract,title_en,assignee_en,abstract_en,label_id
684,6a66121e7fdd6818d5b1f93026f5080d,采集运维动态派工方法,国家电网公司,本发明公开了采集运维动态派工方法，属于电力设备技术领域。目前采集运维工作中不能合理的分配资源...,"Dynamic dispatch method for collection, operat...",State grid Corp.,The invention discloses a dynamic dispatching ...,15
635,5b2a1c586f6258c8228e92ee333eb364,基于金属氧化物纳米带的有序化膜电极及其制备和应用,中国科学院大连化学物理研究所,本发明提供一种基于金属氧化物纳米带的有序化膜电极及其制备方法和应用，包括金属氧化物纳米带的制...,Ordered film electrode based on metal oxide na...,"Dalian Institute of Chemical Physics, Chinese ...",The invention provides an ordered film electro...,13
779,3861b21edb74ac07633f4a79aa873bb0,一种耐腐蚀自清洁石墨烯涂层薄膜及其制备方法,常熟理工学院,本发明公开了一种耐腐蚀自清洁石墨烯涂层薄膜，包括氧化石墨烯和聚二甲基硅氧烷，氧化石墨烯占聚二...,Corrosion resistant self-cleaning graphene coa...,Changshu Institute of Technology,The invention discloses a corrosion-resistant ...,21
2,aaf98d6bfe1932cf1a262812ca59d1ba,一种测试方法及电子设备,腾讯科技(北京)有限公司,本发明提供了一种测试方法及电子设备，该方法包括：基于选取的测试任务确定目标测试用例，根据所述...,Test method and electronic device,"Tencent Technology (Beijing) Co., Ltd.",The invention provides a test method and an el...,0
221,6c16fbaa68fe7e05f57a7aa58bb36065,一种轮胎内侧质量检测装置,江苏托普车轮有限公司,本发明公开了一种轮胎内侧质量检测装置，包括，拔开单元，包括第一拔动组件、第二拔动组件和双向液...,A device for inspecting the inside quality of ...,"Jiangsu Top Wheel Co., Ltd.",The invention discloses a tire inner quality d...,2


In [31]:
#划分数据
eda2=edaData[0:320]

eda3=edaData[321:640]

eda4=edaData[641:957]


In [32]:
#分别使用不同的方法来扩增数据
#extitle,exassignee,exabstract,txtype=getEda(eda1,synonym_replacement)
#edaData['title_en'].extend(extitle)
#edaData['asignee_en'].extend(exassignee)
#edaData['abstract_en'].extend(extitle)
#edaData['label_id'].extend(txtype)
extitle,exassignee,exabstract,txtype=getEda(eda2,random_insertion)
edaData['title_en'].extend(extitle)
edaData['asignee_en'].extend(exassignee)
edaData['abstract_en'].extend(extitle)
edaData['label_id'].extend(txtype)
extitle,exassignee,exabstract,txtype=getEda(eda3,random_swap)
edaData['title_en'].extend(extitle)
edaData['asignee_en'].extend(exassignee)
edaData['abstract_en'].extend(extitle)
edaData['label_id'].extend(txtype)
extitle,exassignee,exabstract,txtype=getEda(eda4,random_deletion)
edaData['title_en'].extend(extitle)
edaData['asignee_en'].extend(exassignee)
edaData['abstract_en'].extend(extitle)
edaData['label_id'].extend(txtype)

UnboundLocalError: ignored

In [ ]:
#封装数据
edaData.to_csv("/content/drive/MyDrive/Colab Notebooks/Academic-paper-classification/dataset/train_en_augMixed.csv")

In [28]:
synonyms = get_synonyms("Soil")
if len(synonyms) >= 1:
  pass
if num_replaced >= 1: 
  pass

sentence = ' '
new_words = sentence.split(' ')

NameError: ignored